In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
# from lxml import etree
import re
from tqdm import tqdm

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# load the supp2021
df = pd.read_csv('supp_names_ui_refer_ui.csv')
df

,supp_names,supp_ui,refer_ui
0,quantum dye macrocyclic europium-chelate,C114158,"*D005063, *D009942"
1,osteoclast activating factor,C008718,*D008222
2,otoline,C008720,*D011108
3,miracil A,C002540,D008154
4,Leakadine,C055240,*D001388
...,...,...,...
317977,calpeptin,C071482,*D004151
317978,"PODXL2 protein, human",C000718228,"*D012795, *D015815"
317979,"At5g20010 protein, Arabidopsis",C490178,"*D016601, *D020931, *D029681"
317980,AMG-714,C000706949,*D061067


In [6]:
supp_list = df['supp_names'].tolist()
supp_list

['quantum dye macrocyclic europium-chelate',
 'osteoclast activating factor',
 'otoline',
 'miracil A',
 'Leakadine',
 'conduritol aziridine',
 '1-oxacephalothin',
 'oxaluric acid',
 '6-oxomestranol-6-(O-carboxymethyl)oxime',
 '2-oxo-3-phenyl-1,2,3,4-tetrahydropyrido(2,3-c)pyrazine',
 '9-oxoprostanoic acid',
 '7-oxo-delta(1)-tetrahydrocannabinol',
 'methyl 6-(1,2,2-trichloroethenyl)imidazo(1,2-a)pyridine-2-carbamate',
 '30-oxyethylated t-octyl phenol formaldehyde tetramer',
 'estane plastic',
 'paracoccidioidin',
 'plutonium hydroxide',
 '3,6-dimethyl-2,5-diaziridinylhydroquinone',
 'pentafluorobenzylimine',
 "3,5,5,O-3',5'-pentamethyl-2'-deoxydihydrouridine",
 'S-pentyl-L-cysteine',
 'peptide PV',
 'phosphatidylkojibiosyl diglyceride',
 'phenacid',
 'peroben',
 'lipid-associated sialic acid',
 'serine containing aminolipid',
 'lonetil M3',
 'lysometra',
 'mastodynon',
 '3-methoxy-abeo-estrone',
 '3 beta-methoxy-dinor-5-cholen-22-ol',
 '20-methylcholesterol',
 '2-O-methyl-D-glucuronic 

In [14]:
pattern = '[A-Z][A-Z]'
acro = []

for name in supp_list:
    if re.findall(pattern, name):
        acro.append(name)
        
acro

['peptide PV',
 'N-phenylacetoaminomethylene-DL-p-nitrophenylalanine',
 'AB 74',
 'DB359',
 'PDZ-RhoGEF, rat',
 'ICM0301B',
 'MAPEP1 protein, Microcystis aeruginosa',
 'TB 68',
 'JBIR-130',
 'JBIR-131',
 'JBIR-132',
 'SB 253514',
 'CMESA-sepharose',
 'SMAD1 protein, human',
 'SMAD2 protein, human',
 'SMAD3 protein, human',
 'STP1 protein, Arabidopsis',
 'SMAD4 protein, human',
 'SMAD5 protein, human',
 'SMAD6 protein, human',
 'SMAD7 protein, human',
 'alanyl-phenylalanyl-psi(CS-N)-prolyl-phenylalanyl-4-nitroanilide',
 'UC 82',
 'MON 0856',
 'MDL 73975',
 'NT020 formula',
 'LY980503',
 'PREP protein, human',
 'HY2 protein, Arabidopsis',
 'SE5-OH',
 'Baypamun HK',
 'transportin SR',
 'transportin SR2',
 'BNC1 protein, human',
 'FZ 560',
 'AH 6696',
 'UP 507-04',
 'beta-lactamase IRT-3',
 'P.GU 1',
 'RFC3 protein, human',
 '264 CP',
 'CN3OP regimen',
 'EX-10-542A',
 'mitochondrial protein mtDBP-C, Xenopus',
 'HSP22.0 protein, Arabidopsis',
 'EC-7012',
 'GH9B18 protein, Arabidopsis',
 'NG

In [15]:
len(acro)

61088

In [17]:
# Make them as new training data set in this format
# {"id": 9, "document_id": "1253656", "ner_tags": ["O", "O", "O", "O", "O", "O", "O", "O", "B-Chemical", "O", "O", "O", "O", "O", "O"], "tokens": ["The", "sperm", "motion", "parameter", "most", "strongly", "associated", "with", "1N", "was", "straight", "-", "line", "velocity", "."], "spans": [[1397, 1400], [1401, 1406], [1407, 1413], [1414, 1423], [1424, 1428], [1429, 1437], [1438, 1448], [1449, 1453], [1454, 1456], [1457, 1460], [1461, 1469], [1469, 1470], [1470, 1474], [1475, 1483], [1483, 1484]]}
# id starts at 18910
# document_id start at 6032168
# span starts at 36022

import json
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize

i = 18910
document_id = 6032168
span_0 = 36022

def tokenize(name):
    if '-' in name:
        t = name.split('-')
        tokens = ['-'] * (len(t) * 2 - 1)
        tokens[0::2] = t
        
        res = []
        for token in tokens:
            res += word_tokenize(token)
        tokens = res
   
    else:       
        tokens = word_tokenize(name)
    
    return tokens

def get_spans(tokens, span_0):
    
    spans = []

    for t in tokens:
        if t not in [',', '-']:
            spans += [[span_0, span_0 + len(t)]]
            span_0 = span_0 + len(t) + 1
        else:
            spans += [[span_0 - 1, span_0]]
            span_0 += 1
    
    return spans

acro_list = []

for name in acro:
    dictionary = {}
    dictionary["id"] = i
    dictionary['document_id'] = document_id
    dictionary['ner_tags'] = ["B-Chemical" if i == 0 else "I-Chemical" for i, t in enumerate(tokenize(name))]
    dictionary['tokens'] = tokenize(name)
    dictionary['spans'] = get_spans(dictionary['tokens'], span_0)
    
    i += 1
    document_id += 1
    span_0 = dictionary['spans'][-1][-1] + 1

    acro_list.append(dictionary)

# acro_list    

In [18]:
filler = {"id": 79998, "document_id": 6093256, "ner_tags": ["O", "O", "O"], "tokens": ["Materials", "and", "Methods"], "spans": [[1463359, 1463368], [1463369, 1463372], [1463373, 1463380]]}

In [19]:
acro_list.append(filler)

In [20]:
len(acro_list)

61089

In [ ]:
with open('add_train_1013.json', 'w') as f:
#     f.write(json.dumps(i) for i in acro_list[:10] + '\n')
    for d in acro_list:
        f.write(json.dumps(d))
        f.write('\n')

In [13]:
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize
word_tokenize(acro[4])

[nltk_data] Downloading package punkt to /Users/wenmosun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

['PDZ-RhoGEF', ',', 'rat']

In [15]:
import spacy
import en_core_web_sm

In [16]:
nlp = en_core_web_sm.load()

nlp(acro[4])

PDZ-RhoGEF, rat

In [23]:
acro[:100]
# do not split '.' as there is not period in this set of data.

for name in acro[:100]:
    if '-' in name:
        t = name.split('-')
        tokens = ['-'] * (len(t) * 2 - 1)
        tokens[0::2] = t
        
        res = []
        for token in tokens:
            res += word_tokenize(token)
        tokens = res
   
    else:       
        tokens = word_tokenize(name)
    
    tokens

['peptide PV',
 'N-phenylacetoaminomethylene-DL-p-nitrophenylalanine',
 'AB 74',
 'DB359',
 'PDZ-RhoGEF, rat',
 'ICM0301B',
 'MAPEP1 protein, Microcystis aeruginosa',
 'TB 68',
 'JBIR-130',
 'JBIR-131',
 'JBIR-132',
 'SB 253514',
 'CMESA-sepharose',
 'SMAD1 protein, human',
 'SMAD2 protein, human',
 'SMAD3 protein, human',
 'STP1 protein, Arabidopsis',
 'SMAD4 protein, human',
 'SMAD5 protein, human',
 'SMAD6 protein, human',
 'SMAD7 protein, human',
 'alanyl-phenylalanyl-psi(CS-N)-prolyl-phenylalanyl-4-nitroanilide',
 'UC 82',
 'MON 0856',
 'MDL 73975',
 'NT020 formula',
 'LY980503',
 'PREP protein, human',
 'HY2 protein, Arabidopsis',
 'SE5-OH',
 'Baypamun HK',
 'transportin SR',
 'transportin SR2',
 'BNC1 protein, human',
 'FZ 560',
 'AH 6696',
 'UP 507-04',
 'beta-lactamase IRT-3',
 'P.GU 1',
 'RFC3 protein, human',
 '264 CP',
 'CN3OP regimen',
 'EX-10-542A',
 'mitochondrial protein mtDBP-C, Xenopus',
 'HSP22.0 protein, Arabidopsis',
 'EC-7012',
 'GH9B18 protein, Arabidopsis',
 'NG

['peptide', 'PV']

['N',
 '-',
 'phenylacetoaminomethylene',
 '-',
 'DL',
 '-',
 'p',
 '-',
 'nitrophenylalanine']

['AB', '74']

['DB359']

...

['AM132']

['NXNL1', 'protein', ',', 'human']

['CED', '-', '5', 'protein', ',', 'C', 'elegans']

['Fuji', 'VII']

['BRCA1', 'protein', ',', 'human']

In [24]:
tokens

['BRCA1', 'protein', ',', 'human']

In [32]:
spans = []
span_0 = 36022

for t in tokens:
    if t not in [',', '-']:
        spans += [[span_0, span_0 + len(t)]]
        span_0 = span_0 + len(t) + 1
    else:
        spans += [[span_0 - 1, span_0]]
        span_0 += 1
    

spans

[[36022, 36027], [36028, 36035], [36035, 36036], [36037, 36042]]

In [2]:
filler = {"id": 79998, "document_id": 6093256, "ner_tags": ["O", "O", "O"], "tokens": ["Materials", "and", "Methods"], "spans": [[1463359, 1463368], [1463369, 1463372], [1463373, 1463380]]}

### Reduce data to drugs with consecutive 4 uppercase letters

In [7]:
pattern_4 = '[A-Z][A-Z][A-Z][A-Z]'
acro_4 = []

for name in supp_list:
    if re.findall(pattern_4, name):
        acro_4.append(name)
        
acro_4

['MAPEP1 protein, Microcystis aeruginosa',
 'JBIR-130',
 'JBIR-131',
 'JBIR-132',
 'CMESA-sepharose',
 'SMAD1 protein, human',
 'SMAD2 protein, human',
 'SMAD3 protein, human',
 'SMAD4 protein, human',
 'SMAD5 protein, human',
 'SMAD6 protein, human',
 'SMAD7 protein, human',
 'PREP protein, human',
 'IMAP protocol',
 'HEOM',
 'ALDH3A1 protein, human',
 'FITC-tyramide',
 'HTLV-II(G12) TaxII protein (337-356)',
 'disulfide reductase (NADH)',
 'SYCRP1 protein, Synechocystis',
 'CSEA protein, E coli',
 'DONS',
 'MIRN517 microRNA, human',
 'MIRN519 microRNA, human',
 'APBA1 protein, human',
 'RUNX1 protein, human',
 'NARG2 protein, mouse',
 'NXNL1 protein, human',
 'BRCA1 protein, human',
 'herpes simplex virus type 2 protein ICSP 11-12',
 'FARP1 protein, mouse',
 'ACLA protein, Dictyostelium discoideum',
 'FANCC protein, human',
 'PXMP2 protein, human',
 'BLAB3 protein, Flavobacterium meningosepticum',
 'cinnamoyl-LIGRLO-amide',
 'NUDC protein, human',
 'GKPV peptide',
 'FGFBP1 protein, h

In [8]:
len(acro_4)

17871

In [9]:
# Make them as new training data set in this format
# {"id": 9, "document_id": "1253656", "ner_tags": ["O", "O", "O", "O", "O", "O", "O", "O", "B-Chemical", "O", "O", "O", "O", "O", "O"], "tokens": ["The", "sperm", "motion", "parameter", "most", "strongly", "associated", "with", "1N", "was", "straight", "-", "line", "velocity", "."], "spans": [[1397, 1400], [1401, 1406], [1407, 1413], [1414, 1423], [1424, 1428], [1429, 1437], [1438, 1448], [1449, 1453], [1454, 1456], [1457, 1460], [1461, 1469], [1469, 1470], [1470, 1474], [1475, 1483], [1483, 1484]]}
# id starts at 18910
# document_id start at 6032168
# span starts at 36022

import json
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize

i = 18910
document_id = 6032168
span_0 = 36022

def tokenize(name):
    if '-' in name:
        t = name.split('-')
        tokens = ['-'] * (len(t) * 2 - 1)
        tokens[0::2] = t
        
        res = []
        for token in tokens:
            res += word_tokenize(token)
        tokens = res
   
    else:       
        tokens = word_tokenize(name)
    
    return tokens

def get_spans(tokens, span_0):
    
    spans = []

    for t in tokens:
        if t not in [',', '-']:
            spans += [[span_0, span_0 + len(t)]]
            span_0 = span_0 + len(t) + 1
        else:
            spans += [[span_0 - 1, span_0]]
            span_0 += 1
    
    return spans

acro_list = []

for name in acro_4:
    dictionary = {}
    dictionary["id"] = i
    dictionary['document_id'] = document_id
    dictionary['ner_tags'] = ["B-Chemical" if i == 0 else "I-Chemical" for i, t in enumerate(tokenize(name))]
    dictionary['tokens'] = tokenize(name)
    dictionary['spans'] = get_spans(dictionary['tokens'], span_0)
    
    i += 1
    document_id += 1
    span_0 = dictionary['spans'][-1][-1] + 1

    acro_list.append(dictionary)

# acro_list    

In [11]:
acro_list[-1]

{'id': 36780,
 'document_id': 6050038,
 'ner_tags': ['B-Chemical', 'I-Chemical', 'I-Chemical', 'I-Chemical'],
 'tokens': ['PODXL2', 'protein', ',', 'human'],
 'spans': [[449347, 449353],
  [449354, 449361],
  [449361, 449362],
  [449363, 449368]]}

In [13]:
acro_list.append({"id": 36781, "document_id": 6050038, "ner_tags": ["O", "O", "O"], "tokens": ["Materials", "and", "Methods"], "spans": [[449369, 449378], [449379, 449382], [449383, 449390]]})

In [14]:
len(acro_list)

17872

In [ ]:
with open('add_train_1015.json', 'w') as f:
#     f.write(json.dumps(i) for i in acro_list[:10] + '\n')
    for d in acro_list:
        f.write(json.dumps(d))
        f.write('\n')